In [110]:
from spatialmath import *
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
%matplotlib widget 

# Demonstrating the spatial math classes

## Working in 3D

### Rotation

Rotations in 3D can be represented by rotation matrices – 3x3 orthonormal matrices – which belong to the group SO(3). 

We can create such a matrix, a rotation of $\pi/4$ radians around the x-axis by

In [111]:
R = SO3.Rx(math.pi/4)

which is an object of type

In [112]:
type(R)

spatialmath.pose3d.SO3

which contains a numpy array which is the SO(3) matrix. We can display its value

In [113]:
R

   1         0         0          
   0         0.707107 -0.707107   
   0         0.707107  0.707107   

which is colored if the console supports color.

We can _compose_ these rotations in an obvious way using standard Python operators

In [114]:
R*R

   1         0         0          
   0         0        -1          
   0         1         0          

which is a rotation by $\pi/4$ _then_ a rotation by $\pi/4$ which is a total of $\pi/2$.  Just to check

In [115]:
SO3.Rx(math.pi/2)

   1         0         0          
   0         0        -1          
   0         1         0          

We could also use the exponentiation operator

In [116]:
R**2

   1         0         0          
   0         0        -1          
   0         1         0          

We could have specified the angle in degrees

In [117]:
SO3.Rx(45, 'deg')
R

   1         0         0          
   0         0.707107 -0.707107   
   0         0.707107  0.707107   

We can visualize what this looks like by

In [118]:
fig = plt.figure() # create a new figure
R.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Click on the coordinate frame and use the mouse to change the viewpoint.

Often we need to describe more complex orientations and we typically use a _3 angle_ convention to do this.  Euler's rotation theorem says that any orientation can be expressed in terms of three rotations about different axes.  

One common convention is roll-pitch-yaw angles

In [119]:
R = SO3.RPY([10, 20, 30], unit='deg')
R

   0.813798 -0.44097   0.378522   
   0.469846  0.882564  0.0180283  
  -0.34202   0.163176  0.925417   

which says that we rotate by 30&deg; about the Z-axis (yaw), _then_ 20&deg; about the Y-axis (pitch) and _then_ 10&deg; about the X-axis – this is the ZYX roll-pitch yaw convention.  We can visualize the resulting orientation.

In [120]:
plt.figure() # create a new figure
R.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

and we can convert any rotation matrix back to its 3-angle representation

In [121]:
R.rpy

array([10., 20., 30.])

#### Constructors

The default constructor yields a null rotation

In [122]:
SO3()

   1         0         0          
   0         1         0          
   0         0         1          

but class supports a number of variant constructors using static methods.  `SO3.Rx()` shown above is one of these but there are also

| Constructor   |  rotation |
|---------------|-----------|
| SO3.Rx(theta)  |  about X-axis |
| SO3.Ry(theta)  |  about Y-axis|
| SO3.Rz(theta)  |  about Z-axis|
| SO3.RPY(rpy)  |  from roll-pitch-yaw angle vector|
| SO3.Eul(euler)  | from Euler angle vector |
| SO3.AngVec(theta, v)  | from rotation and axis |
| SO3.OA  | from orientation and approach vectors |

Imagine we want a rotation that describes a frame that has its y-axis (o-vector) pointing in the world negative z-axis direction and its z-axis (a-vector) pointing in the world x-axis direction

In [123]:
SO3.OA(o=[0,0,-1], a=[1,0,0])

   0         0         1          
  -1         0         0          
   0        -1         0          

We can redo our earlier example using the explicit angle-axis notation

In [124]:
SO3.AngVec(math.pi/4, [1,0,0])

   1         0         0          
   0         0.707107 -0.707107   
   0         0.707107  0.707107   

or a more complex example

In [125]:
SO3.AngVec(30, [1,2,3], unit='deg')

   0.875595 -0.381753  0.29597    
   0.420031  0.904304 -0.0762129  
  -0.238552  0.191048  0.952152   

#### Properties

The object has a number of properties, such as the columns which are often written as $[n, o, a]$

In [126]:
R.n

array([ 0.81379768,  0.46984631, -0.34202014])

or its inverse (in this case its transpose)

In [127]:
R.inv

   0.813798  0.469846 -0.34202    
  -0.44097   0.882564  0.163176   
   0.378522  0.0180283 0.925417   

the shape of the underlying matrix

In [128]:
R.shape

(3, 3)

and the order

In [129]:
R.N

3

indicating it operates in 3D space.

#### Predicates

We can check various properties

In [130]:
[R.isSE, R.isSO, R.isrot(), R.ishom(), R.isrot2(), R.ishom2()]

[False, True, True, False, False, False]

### Representing position

In robotics we also need to describe the position of objects and we can do this with a _homogeneous transformation_ matrix – a 4x4 matrix – which belong to the group SE(3).

We can create such a matrix, for a translation of 1 in the x-direction, 2 in the y-direction and 3 in the z-direction by

In [131]:
T = SE3(1, 2, 3)
T

   1         0         0         1          
   0         1         0         2          
   0         0         1         3          
   0         0         0         1          

which is displayed in a color coded fashion: rotation matrix in red, translation vector in blue, and the constant bottom row in grey.

but class supports a number of variant constructors using static methods.  `SO3.Rx()` shown above is one of these but there are also

| Constructor   |  motion |
|---------------|-----------|
| SE3.Tx(d)  |  translation along X-axis |
| SE3.Ty(d)  |  translation along Y-axis |
| SE3.Tz(d)  |  translation along Z-axis |
| SE3.Rx(theta)  |  rotation about X-axis |
| SE3.Ry(theta)  |  rotation about Y-axis|
| SE3.Rz(theta)  |  rotation about Z-axis|
| SE3.RPY(rpy)  |  rotation from roll-pitch-yaw angle vector|
| SE3.Eul(euler)  | rotation from Euler angle vector |
| SE3.AngVec(theta, v)  | rotation from rotation and axis |
| SE3.OA(ovec, avec)  | rotation from orientation and approach vectors |

We can visualize this as well

In [239]:
plt.figure() # create a new figure
T.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

AttributeError: 'list' object has no attribute 'shape'

### Representing pose

Clearly representing an orientation with 9 numbers is inefficient, and representing 3 translation values with a total of 16 numbers is even more wasteful.  But there's some serious magic possible

In [133]:
T = SE3(1, 2, 3) * SE3.Rx(30, 'deg')
T

   1         0         0         1          
   0         0.866025 -0.5       2          
   0         0.5       0.866025  3          
   0         0         0         1          

Is a composition of two motions: a translation and _then_ a rotation.  We can see the rotation matrix, computed above, in the top-left corner and the translation components in the right-most column.  In the earlier example Out[20] was simply a null-rotation which is represented by the identity matrix.

The frame now looks like this

In [134]:
plt.figure() # create a new figure
T.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Properties

The object has a number of properties, such as the columns which are often written as $[n, o, a]$

In [135]:
T.n

array([1., 0., 0.])

or its inverse (computed in an efficient manner based on the structure of the matrix)

In [136]:
T.inv

   1         0         0        -1          
   0         0.866025  0.5      -3.23205    
   0        -0.5       0.866025 -1.59808    
   0         0         0         1          

We can extract the rotation matrix as a numpy array

In [137]:
T.R

array([[ 1.       ,  0.       ,  0.       ],
       [ 0.       ,  0.8660254, -0.5      ],
       [ 0.       ,  0.5      ,  0.8660254]])

or the translation vector, as a numpy array

In [138]:
T.t

array([1., 2., 3.])

The shape of the underlying SE(3) matrix is

In [139]:
T.shape

(4, 4)

and the order

In [140]:
T.N

3

indicating it operates in 3D space.

#### Predicates

We can check various properties

In [141]:
[T.isSE, T.isSO, T.isrot(), T.ishom(), T.isrot2(), T.ishom2()]

[True, False, False, True, False, False]

### A couple of important points:

When we compose motions they must be of the same type.  An `SE3` object can represent pure transation, pure rotation or both.  If we wish to compose a translation with a rotation, the rotation must be an `SE3` object with zero translation.

### Transforming points

Consider a point at coordinates (1,1,0) in a new coordinate frame.  In the base frame its coordinate is

In [142]:
T * [1, 1, 0]

array([2.       , 2.8660254, 3.5      ])

where the position vector representing the point, has been premultiplied by the homogeneous transformation `T`. The point has been rotated and translated.

The vector is given here as a list but could also be a numpy array.  If the frame is denoted by {A} then our rotation matrix is ${}^0 \mathbf{T}_A$ so a point ${}^A P$ defined with respect to frame {A} is transformed as ${}^0 P = {}^0 \mathbf{T}_A\,{}^A P$.  Imagine now a set of points defining the vertices of a cube

In [143]:
P = np.array([[-1, 1, 1, -1, -1, 1, 1, -1], [-1, -1, 1, 1, -1, -1, 1, 1], [-1, -1, -1, -1, 1, 1, 1, 1]])
P

array([[-1,  1,  1, -1, -1,  1,  1, -1],
       [-1, -1,  1,  1, -1, -1,  1,  1],
       [-1, -1, -1, -1,  1,  1,  1,  1]])

defined with respect to a rotationg reference frame ${}^A P_i$.  Given a rotation ${}^0 \mathbf{T}_A$ as above, we determine the coordinates of the points in the world frame by ${}^0 P_i = ({}^0 \mathbf{T}_A)^{-1} {}^0 P_i$ which we can do in a single operation

In [144]:
Q = T.inv * P

which we could then plot.

In [145]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs=Q[0], ys=Q[1], zs=Q[2], s=20)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Operators

The classes discussed mimic the behavior the mathematical groups SO(3) and SE(3) which contain matrices of particular structure, they are subsets respectively of the sets of all possible real 3x3 and 4x4 matrices.

The only operations on two elements of the group that also belongs to the group is composition (represented by the `*` operator) and inversion.

In [146]:
T = SE3(1, 2, 3) * SE3.Rx(30, 'deg')
[type(T), type(T.inv), type(T*T)]

[spatialmath.pose3d.SE3, spatialmath.pose3d.SE3, spatialmath.pose3d.SE3]

We can simply the writing of

In [147]:
T2 = SE3(4, 5, 6) * SE3.Ry(-40, 'deg')

T * T2.inv

   0.766044  0         0.642788 -5.9209     
   0.321394  0.866025 -0.383022 -1.31757    
  -0.55667   0.5       0.663414 -1.2538     
   0         0         0         1          

by writing

In [148]:
T / T2

   0.766044  0         0.642788 -5.9209     
   0.321394  0.866025 -0.383022 -1.31757    
  -0.55667   0.5       0.663414 -1.2538     
   0         0         0         1          

The inplace versions of operators are also supported, for example

In [149]:
X = T
X /= T2
X

   0.766044  0         0.642788 -5.9209     
   0.321394  0.866025 -0.383022 -1.31757    
  -0.55667   0.5       0.663414 -1.2538     
   0         0         0         1          

#### Non-group operations

Operations such as addition and subtraction are valid for matrices but not for elements of the group, therefore

In [150]:
T + T2

array([[ 1.76604444,  0.        , -0.64278761,  5.        ],
       [ 0.        ,  1.8660254 , -0.5       ,  7.        ],
       [ 0.64278761,  0.5       ,  1.63206985,  9.        ],
       [ 0.        ,  0.        ,  0.        ,  2.        ]])

yields an array, not an `SE3` object.

In [151]:
2 * T

array([[ 2.        ,  0.        ,  0.        ,  2.        ],
       [ 0.        ,  1.73205081, -1.        ,  4.        ],
       [ 0.        ,  1.        ,  1.73205081,  6.        ],
       [ 0.        ,  0.        ,  0.        ,  2.        ]])

In [152]:
T - 1

array([[ 0.       , -1.       , -1.       ,  0.       ],
       [-1.       , -0.1339746, -1.5      ,  1.       ],
       [-1.       , -0.5      , -0.1339746,  2.       ],
       [-1.       , -1.       , -1.       ,  0.       ]])

## Implicit lists

For many tasks we might want to have a group or sequence of rotations or poses. The obvious solution would be to use a Python list

In [153]:
T = [ SE3.Rx(0), SE3.Rx(0.1), SE3.Rx(0.2), SE3.Rx(0.3), SE3.Rx(0.4), SE3.Rx(0.5)]

but we can actually put this list inside the pose object.  There are a few ways to do this, most obviously

In [154]:
T = SE3( [ SE3.Rx(0), SE3.Rx(0.1), SE3.Rx(0.2), SE3.Rx(0.3), SE3.Rx(0.4), SE3.Rx(0.5)] )

which has the same type as an individual pose object

In [155]:
type(T)

spatialmath.pose3d.SE3

but it has length of six

In [156]:
len(T)

6

and when we display it we can see the individual elements

In [157]:
T

[0] =
   1         0         0         0          
   0         1         0         0          
   0         0         1         0          
   0         0         0         1          
[1] =
   1         0         0         0          
   0         0.995004 -0.0998334 0          
   0         0.0998334 0.995004  0          
   0         0         0         1          
[2] =
   1         0         0         0          
   0         0.980067 -0.198669  0          
   0         0.198669  0.980067  0          
   0         0         0         1          
[3] =
   1         0         0         0          
   0         0.955336 -0.29552   0          
   0         0.29552   0.955336  0          
   0         0         0         1          
[4] =
   1         0         0         0          
   0         0.921061 -0.389418  0          
   0         0.389418  0.921061  0          
   0         0         0         1          
[5] =
   1         0         0         0          
   0         0.8775

and we can slice it in the usual way

In [158]:
T[3]

   1         0         0         0          
   0         0.955336 -0.29552   0          
   0         0.29552   0.955336  0          
   0         0         0         1          

or from start to end in steps of two

In [159]:
T[0:-1:2]

[0] =
   1         0         0         0          
   0         1         0         0          
   0         0         1         0          
   0         0         0         1          
[1] =
   1         0         0         0          
   0         0.980067 -0.198669  0          
   0         0.198669  0.980067  0          
   0         0         0         1          
[2] =
   1         0         0         0          
   0         0.921061 -0.389418  0          
   0         0.389418  0.921061  0          
   0         0         0         1          

We could another pose to the end

In [160]:
T.append( SE3.Rx(0.6) )
len(T)

7

The `SE3` class, like all the classes in the spatialmath package, inherits from the `UserList` class giving it all the methods of a Python list.

We could write the above example more succinctly

In [161]:
T = SE3.Rx( np.linspace(0, 0.5, 6) )
len(T)

6

In [162]:
T[3]

   1         0         0         0          
   0         0.955336 -0.29552   0          
   0         0.29552   0.955336  0          
   0         0         0         1          

Consider another rotation

In [163]:
T2 = SE3.Ry(40, 'deg')

then we can write

In [164]:
A = T * T2
len(A)

6

which has produced a new list where each element of `A` is the `T[i] * T2`.  Similarly

In [165]:
B = T2 * T
len(B)

6

which has produced a new list where each element of `B` is the `T2 * T[i]`.

And perhaps not surprisingly 

In [166]:
C = T * T
len(C)

6

which has produced a new list where each element of `C` is the `T[i] * T[i]`.  We can apply such a sequence to a coordinate vectors as we did earlier

In [167]:
P = T * [0, 1, 0]
P

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [1.        , 0.99500417, 0.98006658, 0.95533649, 0.92106099,
        0.87758256],
       [0.        , 0.09983342, 0.19866933, 0.29552021, 0.38941834,
        0.47942554]])

where each element of `T` has transformed the coordinate vector (0, 1, 0), the results being consecutive columns of the resulting numpy array.

Imagine now that we wanted to display our cube for each value in the list, we simply use a `for` loop

In [168]:
for _T in T:
    Q = _T * P
    # plot(Q)

and we can also these list objects inside list comprehensions

In [169]:
np.array([ _T * [0,1,0] for _T in T])

array([[0.        , 1.        , 0.        ],
       [0.        , 0.99500417, 0.09983342],
       [0.        , 0.98006658, 0.19866933],
       [0.        , 0.95533649, 0.29552021],
       [0.        , 0.92106099, 0.38941834],
       [0.        , 0.87758256, 0.47942554]])

### Unit quaternions for 3D rotation

A quaternion can be considered as a type of complex number or more usefully as an order pair comprising a scalar and a vector.  We can create two quaternions

In [181]:
q1 = Quaternion([1,2,3,4])
q1

1.000000 < 2.000000, 3.000000, 4.000000 >

and

In [184]:
q2 = Quaternion([5,6,7,8])
q2

5.000000 < 6.000000, 7.000000, 8.000000 >

where the scalar is before the angle brackets which enclose the vector part.  Operators allow us to add

In [185]:
q1 + q2

6.000000 < 8.000000, 10.000000, 12.000000 >

subtract

In [186]:
q1 - q2

-4.000000 < -4.000000, -4.000000, -4.000000 >

and to multiply

In [187]:
q1 * q2

-60.000000 < 12.000000, 30.000000, 24.000000 >

which follows the rules of Hamilton multiplication.

Properties allow us to extra the scalar part

In [188]:
q1.s

1

and the vector part

In [190]:
q1.v

array([2, 3, 4])

and we can represent it as a numpy array

In [192]:
q1.vec

array([1, 2, 3, 4])

A quaternion has a conjugate

In [193]:
q1.conj

1.000000 < -2.000000, -3.000000, -4.000000 >

and a norm, which is the magnitude of the equivalent 4-vector 

In [194]:
q1.norm

5.477225575051661

which is also the square root of the scalar part of this product

In [195]:
q1 * q1.conj

30.000000 < 0.000000, 0.000000, 0.000000 >

A pure quaternion is one with a zero scalar part

In [197]:
Quaternion.pure([1, 2, 3])

0.000000 < 1.000000, 2.000000, 3.000000 >

A quaternion with a unit norm is called a unit quaternion and can be used to represent rotation in 3D space.

In [203]:
q1 = UnitQuaternion.Rx(30, 'deg')
q1

0.965926 << 0.258819, 0.000000, 0.000000 >>

the convention is that unit quaternions are denoted with double angle brackets.  The norm, as advertised is indeed one

In [204]:
q1.norm

1.0

We create another unit quaternion

In [224]:
q2 = UnitQuaternion.Ry(-40, 'deg')
q2

0.939693 << 0.000000, -0.342020, 0.000000 >>

The rotations can be composed by quaternion multiplication

In [225]:
q3 = q1 * q2
q3

0.907673 << 0.243210, -0.330366, -0.088521 >>

We can convert a quaternion to a rotation matrix

In [226]:
q3.R

array([[ 0.76604444,  0.        , -0.64278761],
       [-0.3213938 ,  0.8660254 , -0.38302222],
       [ 0.5566704 ,  0.5       ,  0.66341395]])

which yields exactly the same answer as if we'd done it using SO(3) rotation matrices

In [229]:
SO3.Rx(30, 'deg') * SO3.Ry(-40, 'deg')

   0.766044  0        -0.642788   
  -0.321394  0.866025 -0.383022   
   0.55667   0.5       0.663414   

The advantages of unit quaternions are that

1. they are compact, just 4 numbers instead of 9
2. multiplication involves fewer operations and is therefore faster
3. numerical errors build up when we multiply rotation matrices together many times, and they lose the structure (the columns are no longer unit length or orthogonal).  Correcting this, the process of _normalization_ is expensive.  For unit quaternions errors will also compound, but normalization is simply a matter of dividing through by the norm

Unit quaternions have an inverse

In [230]:
q2.inv

0.939693 << -0.000000, 0.342020, -0.000000 >>

In [231]:
q1 * q2.inv

0.907673 << 0.243210, 0.330366, 0.088521 >>

or

In [232]:
q1 / q2

0.907673 << 0.243210, 0.330366, 0.088521 >>

We can convert to an SO3 object if we wish

In [235]:
q1.SO3

   1         0         0          
   0         0.866025 -0.5        
   0         0.5       0.866025   

A unit quaternion is not a minimal representation. Since we know the magnitude is 1, then with any 3 elements we can compute the fourth upto a sign ambiguity. 

In [236]:
q1.vec3

array([0.25881905, 0.        , 0.        ])

In [237]:
a = UnitQuaternion.qvmul( q1.vec3, q2.vec3)
a

array([ 0.24321035, -0.33036609, -0.08852133])

from which we can recreate the unit quaternion

In [238]:
UnitQuaternion.Vec3(a)

0.907673 << 0.243210, -0.330366, -0.088521 >>

This is often used in SLAM and bundle adjustment algorithms since it is compact and better behaved than using roll-pitch-yaw or Euler angles.

## Working in 2D

Things are actually much simpler in 2D.  There's only one possible rotation which is around an axis perpendicular to the plane (where the z-axis would have been if it were in 3D).

Rotations in 2D can be represented by rotation matrices – 2x2 orthonormal matrices – which belong to the group SO(2). Just as for the 3D case these matrices have special properties, each column (and row) is a unit vector, and they are all orthogonal, the inverse of this matrix is equal to its transpose, and its determinant is +1.

We can create such a matrix, a rotation of $\pi/4$ radians by

In [171]:
R = SO2(math.pi/4)
R

   0.707107 -0.707107   
   0.707107  0.707107   

or in degrees

In [174]:
SO2(45, unit='deg')

   0.707107 -0.707107   
   0.707107  0.707107   

and we can plot this on the 2D plane

In [175]:
plt.figure() # create a new figure
R.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Once again, it's useful to describe the position of things and we do this this with a homogeneous transformation matrix – a 3x3 matrix – which belong to the group SE(2).

In [176]:
T = SE2(1, 2)
T

AttributeError: 'super' object has no attribute 'arghandler'

which has a similar structure to the 3D case.  The rotation matrix is in the top-left corner and the translation components are in the right-most column.

We can also call the function with the element in a list

In [177]:
T = SE2([1, 2])

AttributeError: 'super' object has no attribute 'arghandler'

In [38]:
plt.figure() # create a new figure
T.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
T = SE2(1, 2) @ SE2(45, 'deg')
T

array([[ 0.70710678, -0.70710678,  1.        ],
       [ 0.70710678,  0.70710678,  2.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [40]:
plt.figure() # create a new figure
T.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …